In [2]:
import pandas as pd
from datetime import datetime, timedelta
pd.set_option('display.max_columns', None)

In [3]:
df_stops = pd.read_csv('data/stops.txt')
df_routes = pd.read_csv('data/routes.txt')
df_stoptimes = pd.read_csv('data/stop_times.txt')
df_trips = pd.read_csv('data/trips.txt')
df_shapes = pd.read_csv('data/shapes.txt')
df_dates = pd.read_csv('data/calendar_dates.txt')

In [4]:
columns_to_drop = ['stop_headsign', 'stop_code', 'route_long_name', 'route_desc', 'route_color', 'route_text_color', 'trip_short_name']

In [5]:
result = pd.merge(df_stoptimes, df_stops, how='inner',on='stop_id')
result = pd.merge(result, df_trips, how='inner',on='trip_id')
result = pd.merge(result, df_routes, how='inner',on='route_id')
result = pd.merge(result, df_dates, how='inner',on='service_id')

In [6]:
result = result.drop(columns=columns_to_drop)

In [7]:
result['arrival_time_minutes'] = result['arrival_time'].map(lambda x: int(x[3:5]))
result['arrival_time_hours'] = result['arrival_time'].map(lambda x: int(x[:2]))
result['departure_time_minutes'] = result['departure_time'].map(lambda x: int(x[3:5]))
result['departure_time_hours'] = result['departure_time'].map(lambda x: int(x[:2]))

result['arrival_time_hours'] = result['arrival_time_hours'].apply(lambda x: x-24 if x >= 24 else x )
result['departure_time_hours'] = result['departure_time_hours'].apply(lambda x: x-24 if x >= 24 else x )

result.date = result.date.astype('str')
result['month'] = result['date'].map(lambda x: x[4:6])
result['day'] = result['date'].map(lambda x: x[6:8])
result['year'] = result['date'].map(lambda x: x[0:4])
result['date2'] = result['year'] + '-' +  result['month'] + '-' + result['day']

result['departure_time'] = result['date2'] + ' ' + result['departure_time']
result['arrival_time'] = result['date2'] + ' ' + result['arrival_time']

result['departure_time'] = result['departure_time'].apply(lambda x: x[:8] + (str(int(x[8:10]) + 1) if int(x[11:13]) >= 24 else x) + x[10:11] + str(int(x[11:13]) - 24) + x[13:] if int(x[11:13]) >= 24 else x)
result['arrival_time'] = result['arrival_time'].apply(lambda x: x[:8] + (str(int(x[8:10]) + 1) if int(x[11:13]) >= 24 else x) + x[10:11] + str(int(x[11:13]) - 24) + x[13:] if int(x[11:13]) >= 24 else x)

result['arrival_time'] = result['arrival_time'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
result['departure_time'] = result['departure_time'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))

<h1>Functions</h1>

In [32]:
def get_all_departures_from_bus_stop(stop_id: int, arrival_time):
#     time = datetime.strptime(arrival_time, '%Y-%m-%d %H:%M')
    time = arrival_time
    all_deparutres = result[(result["stop_id"]==stop_id) &
       (result['departure_time'] >= time) & (result['departure_time'] <= time + timedelta(hours=2)) &
       ~((result['drop_off_type'] == 0) & (result['pickup_type'] == 1))]
    return all_deparutres
#df.sort_values(by=['col1', 'col2'])

In [33]:
def trip_stops(trip_id, stop_id):
    x = pd.DataFrame(result[result['trip_id'] == trip_id].sort_values(by=['stop_sequence']))
    x = x[x['stop_sequence'] >= x[x['stop_id'] == stop_id]['stop_sequence'].values[0]]
    stops = pd.concat([x['route_id'], x['stop_id'],x['stop_name'],x['stop_lat'],x['stop_lon'],x['departure_time'],x['arrival_time']],axis = 1)
    return stops


In [71]:
def get_fastest_busses_from_bus_stop(stop_id: int, time: datetime):
        tmp = get_all_departures_from_bus_stop(stop_id, time)
        trip_idser = tmp.sort_values(by=['departure_time'])['trip_id'].iloc[:6].reset_index(drop=True)
        num = trip_idser.apply(lambda x: x.split('_')[2].split('-')[0])
        bus = set()
        index_to_drop = []
        print(trip_idser)
        for i in range(len(num)):
            if num[i] in bus:
                index_to_drop.append(i)
            bus.add(num[i])
        trip_idser.drop(index_to_drop, inplace=True).reset_index(inplace=True, drop=True)
        print(trip_idser)
        a = pd.DataFrame()
        for i in range(0, len(trip_idser)):
            a = pd.concat([a, trip_stops(trip_idser[i], stop_id)])
        return a

<h1>Tests</h1>

In [72]:
time = datetime.strptime('2023-01-13 15:45', '%Y-%m-%d %H:%M')
c = get_fastest_busses_from_bus_stop(14563, time)

0    158202301131541_12_158-02
1    158202301131601_12_158-03
2    158202301131621_12_158-34
3    158202301131641_12_158-01
4    258202301131700_11_258-31
5    158202301131721_12_158-32
Name: trip_id, dtype: object


AttributeError: 'NoneType' object has no attribute 'reset_index'

In [41]:
c

""


In [10]:
result[(result['drop_off_type'] == 0) & (result['pickup_type'] == 1)]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,stop_name,stop_lat,stop_lon,route_id,service_id,trip_headsign,direction_id,shape_id,wheelchair_accessible,agency_id,route_short_name,route_type,date,exception_type,arrival_time_minutes,arrival_time_hours,departure_time_minutes,departure_time_hours,month,day,year,date2
43,401202301122334_152_401-01,2023-01-13 00:43:00,2023-01-13 00:43:00,14555,43,1,0,Osowa PKP 01,54.420956,18.452618,401,120230112,Osowa PKP,1,401_870029,1,0,N1,700,20230112,1,43,0,43,0,01,12,2023,2023-01-12
45,401202301122258_301_401-01,2023-01-12 23:25:00,2023-01-12 23:25:00,195,22,1,0,Jaworzniaków 01,54.318041,18.568734,401,120230112,Jaworzniaków,0,401_870024,1,0,N1,700,20230112,1,25,23,25,23,01,12,2023,2023-01-12
68,401202301130113_11_401-01,2023-01-13 02:25:00,2023-01-13 02:25:00,195,49,1,0,Jaworzniaków 01,54.318041,18.568734,401,120230112,Jaworzniaków,0,401_870017,1,0,N1,700,20230112,1,25,2,25,2,01,12,2023,2023-01-12
161,401202301130230_392_401-01,2023-01-13 03:43:00,2023-01-13 03:43:00,14555,47,1,0,Osowa PKP 01,54.420956,18.452618,401,120230112,Osowa PKP,1,401_870018,1,0,N1,700,20230112,1,43,3,43,3,01,12,2023,2023-01-12
190,401202301130357_322_401-04,2023-01-13 04:41:00,2023-01-13 04:41:00,1910,29,1,0,Grudziądzka 04,54.396338,18.622572,401,120230112,Grudziądzka,1,401_870033,1,0,N1,700,20230112,1,41,4,41,4,01,12,2023,2023-01-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2051133,4202301261501_42_004-12,2023-01-26 15:57:00,2023-01-26 15:57:00,201,36,1,0,Jelitkowo 02,54.423228,18.592695,4,220230126,Jelitkowo,1,4_884464,1,0,4,900,20230126,1,57,15,57,15,01,26,2023,2023-01-26
2051170,4202301260901_42_004-14,2023-01-26 09:57:00,2023-01-26 09:57:00,201,36,1,0,Jelitkowo 02,54.423228,18.592695,4,220230126,Jelitkowo,1,4_884464,1,0,4,900,20230126,1,57,9,57,9,01,26,2023,2023-01-26
2051207,4202301261121_42_004-14,2023-01-26 12:17:00,2023-01-26 12:17:00,201,36,1,0,Jelitkowo 02,54.423228,18.592695,4,220230126,Jelitkowo,1,4_884464,1,0,4,900,20230126,1,17,12,17,12,01,26,2023,2023-01-26
2051244,4202301261341_42_004-14,2023-01-26 14:37:00,2023-01-26 14:37:00,201,36,1,0,Jelitkowo 02,54.423228,18.592695,4,220230126,Jelitkowo,1,4_884464,1,0,4,900,20230126,1,37,14,37,14,01,26,2023,2023-01-26


In [11]:
tmp = get_all_departures_from_bus_stop(14563,'2023-01-13 00:45')
trip_idser = tmp.sort_values(by=['departure_time'])['trip_id'].iloc[:6].reset_index(drop=True)

tmp.sort_values(by=['departure_time'])

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,stop_name,stop_lat,stop_lon,route_id,service_id,trip_headsign,direction_id,shape_id,wheelchair_accessible,agency_id,route_short_name,route_type,date,exception_type,arrival_time_minutes,arrival_time_hours,departure_time_minutes,departure_time_hours,month,day,year,date2
157072,158202301130446_22_158-02,2023-01-13 04:49:00,2023-01-13 04:49:00,14563,1,0,0,Rondo Mazowieckiego 01,54.388469,18.648582,158,120230113,Stogi,1,158_903692,1,0,158,700,20230113,1,49,4,49,4,01,13,2023,2023-01-13
157284,158202301130506_22_158-03,2023-01-13 05:09:00,2023-01-13 05:09:00,14563,1,0,0,Rondo Mazowieckiego 01,54.388469,18.648582,158,120230113,Stogi,1,158_903692,1,0,158,700,20230113,1,9,5,9,5,01,13,2023,2023-01-13
158902,258202301130500_11_258-10,2023-01-13 05:18:00,2023-01-13 05:18:00,14563,11,0,0,Rondo Mazowieckiego 01,54.388469,18.648582,258,120230113,Stogi,0,258_903513,1,0,258,700,20230113,1,18,5,18,5,01,13,2023,2023-01-13
158878,258202301130515_11_258-01,2023-01-13 05:33:00,2023-01-13 05:33:00,14563,11,0,0,Rondo Mazowieckiego 01,54.388469,18.648582,258,120230113,Stogi,0,258_903513,1,0,258,700,20230113,1,33,5,33,5,01,13,2023,2023-01-13
157520,158202301130557_22_158-12,2023-01-13 06:00:00,2023-01-13 06:00:00,14563,1,0,0,Rondo Mazowieckiego 01,54.388469,18.648582,158,120230113,Stogi,1,158_903692,1,0,158,700,20230113,1,0,6,0,6,01,13,2023,2023-01-13
158830,258202301130720_11_258-12,2023-01-13 07:40:00,2023-01-13 07:40:00,14563,11,0,0,Rondo Mazowieckiego 01,54.388469,18.648582,258,120230113,Stogi,0,258_903513,1,0,258,700,20230113,1,40,7,40,7,01,13,2023,2023-01-13
157117,158202301130737_12_158-02,2023-01-13 07:57:00,2023-01-13 07:57:00,14563,11,0,0,Rondo Mazowieckiego 01,54.388469,18.648582,158,120230113,Stogi,1,158_903464,1,0,158,700,20230113,1,57,7,57,7,01,13,2023,2023-01-13
157352,158202301131015_12_158-03,2023-01-13 10:35:00,2023-01-13 10:35:00,14563,11,0,0,Rondo Mazowieckiego 01,54.388469,18.648582,158,120230113,Stogi,1,158_903464,1,0,158,700,20230113,1,35,10,35,10,01,13,2023,2023-01-13
157745,158202301131055_12_158-01,2023-01-13 11:15:00,2023-01-13 11:15:00,14563,11,0,0,Rondo Mazowieckiego 01,54.388469,18.648582,158,120230113,Stogi,1,158_903464,1,0,158,700,20230113,1,15,11,15,11,01,13,2023,2023-01-13
157162,158202301131135_12_158-02,2023-01-13 11:55:00,2023-01-13 11:55:00,14563,11,0,0,Rondo Mazowieckiego 01,54.388469,18.648582,158,120230113,Stogi,1,158_903464,1,0,158,700,20230113,1,55,11,55,11,01,13,2023,2023-01-13


In [12]:
def trip_stops(trip_id, stop_id):
    x = pd.DataFrame(result[result['trip_id'] == trip_id].sort_values(by=['stop_sequence']))
    x = x[x['stop_sequence'] >= x[x['stop_id'] == stop_id]['stop_sequence'].values[0]]
    stops = pd.concat([x['route_id'], x['stop_id'],x['stop_name'],x['stop_lat'],x['stop_lon'],x['departure_time'],x['arrival_time']],axis = 1)
    return stops


In [13]:
trip_id = trip_idser[2]
stop_id = 14563
x = pd.DataFrame(result[result['trip_id'] == trip_id].sort_values(by=['stop_sequence']))
x[x['stop_sequence'] >= x[x['stop_id'] == stop_id]['stop_sequence'].values[0]]
# x[x['stop_id'] == stop_id]['stop_sequence'].values[0]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,stop_name,stop_lat,stop_lon,route_id,service_id,trip_headsign,direction_id,shape_id,wheelchair_accessible,agency_id,route_short_name,route_type,date,exception_type,arrival_time_minutes,arrival_time_hours,departure_time_minutes,departure_time_hours,month,day,year,date2
158902,258202301130500_11_258-10,2023-01-13 05:18:00,2023-01-13 05:18:00,14563,11,0,0,Rondo Mazowieckiego 01,54.388469,18.648582,258,120230113,Stogi,0,258_903513,1,0,258,700,20230113,1,18,5,18,5,01,13,2023,2023-01-13
158903,258202301130500_11_258-10,2023-01-13 05:19:00,2023-01-13 05:19:00,1644,12,0,0,Polsat Plus Arena Gdańsk 03,54.387014,18.648681,258,120230113,Stogi,0,258_903513,1,0,258,700,20230113,1,19,5,19,5,01,13,2023,2023-01-13
158884,258202301130500_11_258-10,2023-01-13 05:23:00,2023-01-13 05:23:00,1072,13,3,3,Ku Ujściu 02,54.377388,18.679429,258,120230113,Stogi,0,258_903513,1,0,258,700,20230113,1,23,5,23,5,01,13,2023,2023-01-13
158890,258202301130500_11_258-10,2023-01-13 05:28:00,2023-01-13 05:28:00,1898,14,3,3,Kontenerowa 01,54.380498,18.696779,258,120230113,Stogi,0,258_903513,1,0,258,700,20230113,1,28,5,28,5,01,13,2023,2023-01-13
158891,258202301130500_11_258-10,2023-01-13 05:30:00,2023-01-13 05:30:00,1900,15,0,0,Terminal DCT 01,54.383071,18.706292,258,120230113,Stogi,0,258_903513,1,0,258,700,20230113,1,30,5,30,5,01,13,2023,2023-01-13
158885,258202301130500_11_258-10,2023-01-13 05:34:00,2023-01-13 05:34:00,1070,16,3,3,Sucharskiego - PKP 02,54.371826,18.683300,258,120230113,Stogi,0,258_903513,1,0,258,700,20230113,1,34,5,34,5,01,13,2023,2023-01-13
158886,258202301130500_11_258-10,2023-01-13 05:36:00,2023-01-13 05:36:00,14591,17,3,3,Andruszkiewicza 02,54.362125,18.690280,258,120230113,Stogi,0,258_903513,1,0,258,700,20230113,1,36,5,36,5,01,13,2023,2023-01-13
158883,258202301130500_11_258-10,2023-01-13 05:37:00,2023-01-13 05:37:00,1692,18,0,0,Sucharskiego 04,54.359630,18.690944,258,120230113,Stogi,0,258_903513,1,0,258,700,20230113,1,37,5,37,5,01,13,2023,2023-01-13
158904,258202301130500_11_258-10,2023-01-13 05:39:00,2023-01-13 05:39:00,14813,19,3,3,Kaczeńce 04,54.363078,18.698770,258,120230113,Stogi,0,258_903513,1,0,258,700,20230113,1,39,5,39,5,01,13,2023,2023-01-13
158905,258202301130500_11_258-10,2023-01-13 05:40:00,2023-01-13 05:40:00,14845,20,3,3,Ugory eMOCja 04,54.362929,18.704151,258,120230113,Stogi,0,258_903513,1,0,258,700,20230113,1,40,5,40,5,01,13,2023,2023-01-13


In [14]:
trip_stops(trip_idser[2], 14563)

,route_id,stop_id,stop_name,stop_lat,stop_lon,departure_time,arrival_time
158902,258,14563,Rondo Mazowieckiego 01,54.388469,18.648582,2023-01-13 05:18:00,2023-01-13 05:18:00
158903,258,1644,Polsat Plus Arena Gdańsk 03,54.387014,18.648681,2023-01-13 05:19:00,2023-01-13 05:19:00
158884,258,1072,Ku Ujściu 02,54.377388,18.679429,2023-01-13 05:23:00,2023-01-13 05:23:00
158890,258,1898,Kontenerowa 01,54.380498,18.696779,2023-01-13 05:28:00,2023-01-13 05:28:00
158891,258,1900,Terminal DCT 01,54.383071,18.706292,2023-01-13 05:30:00,2023-01-13 05:30:00
158885,258,1070,Sucharskiego - PKP 02,54.371826,18.683300,2023-01-13 05:34:00,2023-01-13 05:34:00
158886,258,14591,Andruszkiewicza 02,54.362125,18.690280,2023-01-13 05:36:00,2023-01-13 05:36:00
158883,258,1692,Sucharskiego 04,54.359630,18.690944,2023-01-13 05:37:00,2023-01-13 05:37:00
158904,258,14813,Kaczeńce 04,54.363078,18.698770,2023-01-13 05:39:00,2023-01-13 05:39:00
158905,258,14845,Ugory eMOCja 04,54.362929,18.704151,2023-01-13 05:40:00,2023-01-13 05:40:00


In [15]:
result[result['stop_name'] == 'Rondo Mazowieckiego 01']

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,stop_name,stop_lat,stop_lon,route_id,service_id,trip_headsign,direction_id,shape_id,wheelchair_accessible,agency_id,route_short_name,route_type,date,exception_type,arrival_time_minutes,arrival_time_hours,departure_time_minutes,departure_time_hours,month,day,year,date2
70341,158202301122228_12_158-08,2023-01-12 22:45:00,2023-01-12 22:45:00,14563,11,0,0,Rondo Mazowieckiego 01,54.388469,18.648582,158,120230112,Stogi,1,158_903464,1,0,158,700,20230112,1,45,22,45,22,01,12,2023,2023-01-12
70353,158202301120446_22_158-02,2023-01-12 04:49:00,2023-01-12 04:49:00,14563,1,0,0,Rondo Mazowieckiego 01,54.388469,18.648582,158,120230112,Stogi,1,158_903692,1,0,158,700,20230112,1,49,4,49,4,01,12,2023,2023-01-12
70398,158202301120737_12_158-02,2023-01-12 07:57:00,2023-01-12 07:57:00,14563,11,0,0,Rondo Mazowieckiego 01,54.388469,18.648582,158,120230112,Stogi,1,158_903464,1,0,158,700,20230112,1,57,7,57,7,01,12,2023,2023-01-12
70443,158202301121135_12_158-02,2023-01-12 11:55:00,2023-01-12 11:55:00,14563,11,0,0,Rondo Mazowieckiego 01,54.388469,18.648582,158,120230112,Stogi,1,158_903464,1,0,158,700,20230112,1,55,11,55,11,01,12,2023,2023-01-12
70465,158202301121334_12_158-02,2023-01-12 13:54:00,2023-01-12 13:54:00,14563,11,0,0,Rondo Mazowieckiego 01,54.388469,18.648582,158,120230112,Stogi,1,158_903464,1,0,158,700,20230112,1,54,13,54,13,01,12,2023,2023-01-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150041,258202301260720_11_258-12,2023-01-26 07:40:00,2023-01-26 07:40:00,14563,11,0,0,Rondo Mazowieckiego 01,54.388469,18.648582,258,120230126,Stogi,0,258_904303,1,0,258,700,20230126,1,40,7,40,7,01,26,2023,2023-01-26
1150065,258202301261700_11_258-31,2023-01-26 17:20:00,2023-01-26 17:20:00,14563,11,0,0,Rondo Mazowieckiego 01,54.388469,18.648582,258,120230126,Stogi,0,258_904303,1,0,258,700,20230126,1,20,17,20,17,01,26,2023,2023-01-26
1150089,258202301260515_11_258-01,2023-01-26 05:33:00,2023-01-26 05:33:00,14563,11,0,0,Rondo Mazowieckiego 01,54.388469,18.648582,258,120230126,Stogi,0,258_904303,1,0,258,700,20230126,1,33,5,33,5,01,26,2023,2023-01-26
1150113,258202301260500_11_258-10,2023-01-26 05:18:00,2023-01-26 05:18:00,14563,11,0,0,Rondo Mazowieckiego 01,54.388469,18.648582,258,120230126,Stogi,0,258_904303,1,0,258,700,20230126,1,18,5,18,5,01,26,2023,2023-01-26


In [26]:
result[(result['route_id'] == 403) & (result['trip_id'].apply(lambda x: x.split('_')[1]) == 62)]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,stop_name,stop_lat,stop_lon,route_id,service_id,trip_headsign,direction_id,shape_id,wheelchair_accessible,agency_id,route_short_name,route_type,date,exception_type,arrival_time_minutes,arrival_time_hours,departure_time_minutes,departure_time_hours,month,day,year,date2


0          152
1          152
2          152
3          152
4          152
          ... 
2051286     32
2051287     32
2051288     32
2051289     32
2051290     32
Name: trip_id, Length: 2051291, dtype: object

In [21]:
result['trip_id'][0].split('_')[1]

'152'